In [2]:
import uproot
import sys
import glob
import os
import struct
import time
import csv
import pickle
import math
import gc

import numpy                  as np
import pandas                 as pd
import matplotlib             as mpl
import matplotlib.pyplot      as plt
import tensorflow             as tf
from   tensorflow             import keras

import scipy
from   scipy             import interpolate
from   scipy.interpolate import CubicSpline, splev, splrep, PPoly
from   scipy.optimize    import curve_fit, minimize

# from   scipy.stats            import kde
# from   mpl_toolkits           import mplot3d
# from   mpl_toolkits.mplot3d   import Axes3D

# sys.path.insert(0, "/home/chan/2_ROOT/0_FROST/0_FROSTpython")
# from   rbDST2dict import rbDST2dict_chan as frz
# from   rbDST2dict.rbDST2dict_chan import rbDST2dict_chan as frz
# from   rbDST2dict             import rbDST2dict_chan as frz      #rbDST2dict=dir, rbDST2dict.py=module, rbDST2dict()=class
                                                                 #you need to specify the folder -> "from rbDST2dict"

#To increase the cell width + load Images
from IPython.core.display import display, HTML, Image
# display(HTML("<style>.container { width:100% !important; }</style
#Pandas options
pd.options.display.max_columns = None
# pd.options.display.max_rows    = 100

#Plotting Config
mpl.style.use("default")
mpl.rcParams["axes.facecolor"] = "#EAEAF2" 
mpl.rcParams['figure.dpi']     = 100 
mpl.rcParams['savefig.dpi']    = 100 

# ML 

### Target Selection 

#### Training Data (MUST)

###### Notes 

(NOTES)
* As # of butanol training increased (while same # of carbon training events), more events were classified as butanol
  events. $\rightarrow$ Weird carbon peack at ambiguous region disappeared which is good.
  * My guess is that ratio of training data for butanol vs training data for carbon must be same as the ratio of 
    real butanol events vs real carbon events:
    $$ \frac{\text{# butanol train events}}{\text{# carbon train events}} = \frac{\text{# butanol events}}{\text{# carbon events}} $$

###### Code (MUST)

In [32]:
#Butanol
df_butanol_train = df_GMSH[(df_GMSH["z"] >= -3.3) & (df_GMSH["z"] <= 3.3)]
#Carbon
df_carbon_train = df_GMSH[(df_GMSH["z"] >= 5.5) & (df_GMSH["z"] <= 7.0)]
#Polythene
df_polythene_train = df_GMSH[(df_GMSH["z"] >= 15) & (df_GMSH["z"] <= 17.0)]

In [33]:
num_carb_train = 20000
num_buta_train = int(len(df_butanol_train) / len(df_carbon_train) * num_carb_train)
num_poly_train = int(len(df_polythene_train) / len(df_carbon_train) * num_carb_train)

print("# of carbon training events    = ", num_carb_train)
print("# of butanol training events   = ", num_buta_train)
print("# of polythene training events = ", num_poly_train)

# of carbon training events    =  20000
# of butanol training events   =  135899
# of polythene training events =  21887


In [34]:
#Select Random points for reduced training data
df_butanol_train   = df_butanol_train.sample(num_buta_train)
df_carbon_train    = df_carbon_train.sample(num_carb_train)
df_polythene_train = df_polythene_train.sample(num_poly_train)

In [35]:
print("len(df_butanol_train)   = {:d}".format(len(df_butanol_train)))
print("len(df_carbon_train)    = {:d}".format(len(df_carbon_train)))
print("len(df_polythene_train) = {:d}".format(len(df_polythene_train)))

len(df_butanol_train)   = 135899
len(df_carbon_train)    = 20000
len(df_polythene_train) = 21887


In [ ]:
#Training data histrogram (CHECK)
training_hist_TS = plt.figure(figsize=(10,10))
plt.hist(df_butanol_train.z, alpha=0.5, bins=100, histtype='stepfilled', label="Butanol Training", color="#55A868")
plt.hist(df_carbon_train.z, alpha=0.5, bins=100, histtype='stepfilled', label="Carbon Training", color="#C44E52")
plt.hist(df_polythene_train.z, alpha=0.5, bins=100, histtype='stepfilled', label="Polythene Training", color="#4C72B0")

plt.xlabel("Z-Vertex Position (cm)", fontsize=20)
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.show()
training_hist_TS.savefig("training_hist_TS")

In [37]:
#Ground Truth column - train labels
df_butanol_train["truth"]   = 0
df_carbon_train["truth"]    = 1
df_polythene_train["truth"] = 2

In [38]:
#Combine trainig data of butanol + carbon + polythene
df_train_all = pd.concat([df_butanol_train, df_carbon_train, df_polythene_train])

#Separate df for ground truth - train label
df_train_label = pd.DataFrame(df_train_all["truth"])

#Drop groun truth column in training data
df_train_all.drop(columns=["truth"], inplace=True)

In [39]:
df_GMSH.columns

Index(['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'epho', 'x',
       'y', 'z', 'TRIGBITS', 'el_px', 'el_py', 'el_pz', 'el_E', 'timediff',
       'runNum', 'p_abs', 'el_p_abs', 'mmsq_pi0', 'el_mmsq_pi0', 'theta',
       'phi', 'sector', 'tof_pad', 'beta_diff', 'w', 'cthe_cm', 'tar_Pol',
       'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
       'pol_pho_stat', 'T_fin', 'el_pc_p_abs', 'el_pc_mmsq_pi0', 'phi_sec'],
      dtype='object')

In [41]:
#Drop unwanted columns

df_train_all.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'TRIGBITS',
                           'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
                           'sector', 'tof_pad', 'beta_diff', 'w',
                           'cthe_cm', 'tar_Pol',
                           'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
                           'pol_pho_stat'], inplace=True)


# df_train_all.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'scPdHt', 'TRIGBITS',
#                            'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
#                            'sector', 'tof_pad', 'beta_diff', 'w', 'beta_cm',
#                            'gamma_cm', 'pz_cm', 'p_cm_abs', 'theta_cm', 'cthe_cm', 'tar_Pol',
#                            'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
#                            'pol_pho_stat'], inplace=True)

# df_train_all.drop(columns=["pid", "ntrk", "scPdHt", "TRIGBITS", "runNum", "p_abs", 
#                            "beta_diff", "sector", "tof_pad", "beta_cm", "gamma_cm", "pz_cm", "p_cm_abs",
#                            "theta_cm", "cthe_cm", "tar_Pol", "tar_Pol_sign", "bm_helicity", "pol_pho", \
#                            'T_pred', 'ice_pred', 'T_fin', 'sf', 'tar_Pol_stat', 'tar_Pol_sys', 'pol_pho_stat'], inplace=True)

In [ ]:
# #Drop when doing 2nd time
# df_train_all.drop(columns=["T_pred", "ice_pred", "T_fin"], inplace=True)

In [42]:
df_train_all.columns

Index(['epho', 'x', 'y', 'z', 'el_px', 'el_py', 'el_pz', 'el_E', 'el_p_abs',
       'el_mmsq_pi0', 'theta', 'phi', 'T_fin', 'el_pc_p_abs', 'el_pc_mmsq_pi0',
       'phi_sec'],
      dtype='object')

In [43]:
#Convert to numpy ndarray 
train_all   = df_train_all.values
train_label = df_train_label.values 

In [44]:
print(len(train_all))
print(len(train_label))
print(train_all.shape)
print(train_label.shape)

177786
177786
(177786, 16)
(177786, 1)


In [89]:
#Study Correlation of parameters (NONEED)
# df_train_all.corr()

In [90]:
# #Seaborn Heatmap (NONEED)
# sns.set(font_scale=1.5)
# g9a_heatmap = plt.figure(figsize=(13,13))
# #Remove Upper Right Triangle
# mask=np.zeros_like(df_train_all.corr())
# mask[np.triu_indices_from(mask)] = True
# #Set the Upper Right Triangle as white blank spaces
# with sns.axes_style("white"):
#     sns.heatmap(df_train_all.corr(), vmin=-1, vmax=1, mask=mask)
# plt.show()
# g9a_heatmap.savefig("g9a_heatmap")

#### Test Data (MUST)

###### Code (MUST)

In [46]:
#drop the columns that df_train_all dropped already

df_GMSH_ml = df_GMSH.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'TRIGBITS',
                           'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
                           'sector', 'tof_pad', 'beta_diff', 'w',
                           'cthe_cm', 'tar_Pol',
                           'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
                           'pol_pho_stat'])


# df_GMSH_ml = df_GMSH.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'scPdHt', 'TRIGBITS',
#                            'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
#                            'sector', 'tof_pad', 'beta_diff', 'w', 'beta_cm',
#                            'gamma_cm', 'pz_cm', 'p_cm_abs', 'theta_cm', 'cthe_cm', 'tar_Pol',
#                            'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
#                            'pol_pho_stat'])

# df_GMSH_ml = df_GMSH.drop(columns=["pid", "ntrk", "scPdHt", "TRIGBITS", "runNum", "p_abs", 
#                            "beta_diff", "sector", "tof_pad", "beta_cm", "gamma_cm", "pz_cm", "p_cm_abs",
#                            "theta_cm", "cthe_cm", "tar_Pol", "tar_Pol_sign", "bm_helicity", "pol_pho", \
#                            'T_pred', 'ice_pred', 'T_fin', 'sf', 'tar_Pol_stat', 'tar_Pol_sys', 'pol_pho_stat'])


In [47]:
#Change order of columns so that columns of training data and test data matches¶
df_train_all_cols = df_train_all.columns.tolist()
df_GMSH_ml = df_GMSH_ml[df_train_all_cols]

print(df_train_all_cols)
print(df_GMSH_ml.columns.tolist())

['epho', 'x', 'y', 'z', 'el_px', 'el_py', 'el_pz', 'el_E', 'el_p_abs', 'el_mmsq_pi0', 'theta', 'phi', 'T_fin', 'el_pc_p_abs', 'el_pc_mmsq_pi0', 'phi_sec']
['epho', 'x', 'y', 'z', 'el_px', 'el_py', 'el_pz', 'el_E', 'el_p_abs', 'el_mmsq_pi0', 'theta', 'phi', 'T_fin', 'el_pc_p_abs', 'el_pc_mmsq_pi0', 'phi_sec']


In [48]:
#convert df_GPID_MVRT to numpy ndarry
test_all = df_GMSH_ml.values

#### Summary of training + test data 

In [ ]:
# So far we have the following data sets: (NOTES)
# * test_all = test data set 
#   * df_GMSH_ml
# * train_all = training data set
#   * df_train_all 
# * train_label  = trainig label (true values)
#   * df_train_label

#### Build Model + Prediction (keras) (MUST)

###### Notes 

__`Basics of NN`__ (NOTES)
* Each neural netwrork operates on input data as follow:
  $$ output = relu(dot(W, input) + b) $$
  * $b=trainable\ parameter$ are tensors that is attribute of the _bias_
  * $W=weights$ (_coefficients_) - tensor that is an attribute of the _kernel_ 
    * Initially, these weights are filled with small randome values, _random initialization_
    * gradual adjustment of these weights after each iteration in _training loop_ = training the model data
      1. Draw training samples _x_ and corresponding targets _y_
      2. Run the network on _x_ (step called _forward pass_) to obtain predictions _y_pred_
      3. Compute the loss of the network on the batch: a measure of the mismatch between _y_pred_ and _y_.
      4. __Optimizer__: Update all weights of the network in a way that slightly reduces the loss on this batch. 
        * How can you compute how the coefficient should be altered? improved?
          1. Naive solution
            * freeze all weights in the network except the one scalar coefficient being considered and try new values
              untill it improves. 
          2. Better solution (_stochastic gradient descent_ SGD algorithm)
            * Since all tensor operations are _differentiable_, compute _graduent_ of the loss function with regards 
              to the network's coefficients/parameters. Then, move the coefficients in the opposite direction from the
              gradient, thus decreasing the loss: 
              $$ W -= step\ *\ gradient $$
              $$ new W = old W - learning rate * gradient $$
            * step size = learning rate
            * Find the minimum and alter the coefficients toward minimum.
            * _stochastic_ refers to the fact that each batch of data is drawn at random.
          3. SGD with momentum
            * use gradient of not only the current value (current velocity), but also the past value (past velocity). Thus,
              it allows the see local minimum + global minimum.
            
* __Optimizer__ and __loss function__ are what determines the NN algorithm - `where physics can be applied`.
  * I assume most of the optimizers use statistical methods using traing data and computing relations between parameters of
    training data to find a way to best update Weights.
  * However, if you have external knowledge (physics in our case: __`relativistic kinematics`__) about the system that you
    can put as constraints when optimizing, it will yield better results:
    * Scattering angle limitation
      * For example, at certain forbidden angles, angles weigh less than momentum, energy, ... $\rightarrow$ update weights 
        in such a way that the layers use parameters such as momentum, energy , etc more than scattering angles. 


__`Code Explanation`__ (NOTES)
* __keras.Sequential__ model is a linear stack of layers
* __tf.keras.layers.Dense__ 
  * the network will consist of two dense (fully-connected) neural layers. 
    * 1st dense neural layer has 15 nodes since we have 15 parameters 
      * tf.nn.relu = Re(ctified) L(inear) U(nit)
        * As results from each layer needs to yield a specific representation of result (i.e. 0 or 1), there
          for we need to rectify the result using a function and in this case, a linear function.
    * 2nd dense neural layer has 3-node softmax layer - `This is where __prediction__ happens`
      * tf.nn.softmax = For each event, it returns an array of 3 probability scores that sum to 1 - Normalisation of 
        sum(input)=1
        * each node contains a score that indicates the probability that the current event belong to butanol, carbon,
          or polythene.

In [ ]:
# (NOTES)
# * fit() method is to train the model to fit to the data
# * epochs = 5 means you train that 5 times
# * Accruracy increased as more trainings are done

###### Code (MUST)

In [49]:
#Layer Setup
#    -linear staack of layers
#    -Specify input_shape in the first layer if you want to use model.save() function
#        -input_shape = (# of columns, )
model_ts = keras.Sequential()
# model_ts.add(keras.layers.Dense(100, activation=tf.nn.relu))
model_ts.add(keras.layers.Dense(15, activation=tf.nn.relu))
model_ts.add(keras.layers.Dense(15, activation=tf.nn.relu))
# model_ts.add(keras.layers.Dense(15, activation=tf.nn.relu))
model_ts.add(keras.layers.Dense(3, activation=tf.nn.softmax))

In [50]:
#Compile: Optimizer + Loss function
model_ts.compile(optimizer=tf.optimizers.Adam(),
                 loss     = "sparse_categorical_crossentropy",
                 metrics  = ["accuracy"])

In [51]:
#Traing
#    -fit() method is to train the model to fit to the data
#    -epochs = 5 means you train that 5 times
#    -Accruracy increased as more trainings are done
model_ts.fit(train_all, train_label, epochs=3)

Epoch 1/3
5556/5556 [==============================] - 10s 2ms/step - loss: 0.1344 - accuracy: 0.9693
Epoch 2/3
5556/5556 [==============================] - 8s 1ms/step - loss: 0.0018 - accuracy: 0.9996
Epoch 3/3
5556/5556 [==============================] - 7s 1ms/step - loss: 6.4710e-04 - accuracy: 0.9998


In [52]:
#Predict
predictions = model_ts.predict(test_all)

In [53]:
#predictions_list is a list of final predictions of target on test_all data
predictions_list = []
for i in range(0, len(predictions)):
    predictions_list.append(np.argmax(predictions[i]))

In [54]:
#Append predicted values (predictions_list) to df
df_GMSH["T_pred"] = pd.Series(predictions_list, index=df_GMSH.index)

In [ ]:
#Target Prediction (CHECK)
ML_target_sep, [ax1, ax2, ax3] = plt.subplots(3, 1, sharex=True, figsize=(15,15))

#target label
target_label = df_GMSH.T_pred.unique()

for i in target_label:
    hit = df_GMSH[df_GMSH.T_pred == i]
    if i == 0:
        target = "Butanol"
        n_buta, b_buta, p_buta = ax1.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 
    elif i== 1:
        target = "Carbon"
        ax2.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 
    elif i==2:
        target = "Polythene"
        ax3.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 

#Grids
for i in [ax1, ax2, ax3]:
    i.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    i.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    i.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
    i.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")        
        
ax1.legend()
ax2.legend()
ax3.legend()
plt.xlabel("Z-Vertex Position (cm)")
plt.xlim(-10,20)

#Spacing 0
ML_target_sep.subplots_adjust(hspace=0.)

plt.show()
# ML_target_sep.savefig("./run_period_1/ML_target_sep")

In [ ]:
#Target Prediction - on Same axes (CHECK)
ML_TS_pred   = plt.figure(figsize=(15,10))

#Bins
bin_num = int(0.15*np.sqrt(len(df_GMSH)))

#Plot
n_buta, b_buta, p_buta = plt.hist(df_GMSH[df_GMSH["T_pred"]==0]["z"], bins=bin_num, label="Butanol", \
                                  alpha=0.7, color="#55A868")
n_carb, b_carb, p_carb = plt.hist(df_GMSH[df_GMSH["T_pred"]==1]["z"], bins=bin_num, label="Carbon", \
                                  alpha=0.7, color="#C44E52")

#Stats
peak_buta = b_buta[np.argmax(n_buta)]
mean_buta = df_GMSH[df_GMSH["T_pred"]==0]["z"].astype(float).mean()
std_buta  = df_GMSH[df_GMSH["T_pred"]==0]["z"].astype(float).std()
peak_carb = b_carb[np.argmax(n_carb)]
mean_carb = df_GMSH[df_GMSH["T_pred"]==1]["z"].astype(float).mean()
std_carb  = df_GMSH[df_GMSH["T_pred"]==1]["z"].astype(float).std()

#Texts
plt.text(0.03, 0.75, "Butanol \n Peak {:>10.3f} \n $\mu$ {:>15.3f} \n $\sigma$ {:>15.3f}" \
         .format(peak_buta, mean_buta, std_buta), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)
plt.text(0.03, 0.55, "Carbon \n Peak {:>10.3f} \n $\mu$ {:>15.3f} \n $\sigma$ {:>15.3f}" \
         .format(peak_carb, mean_carb, std_carb), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)


#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/10000)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))
        
plt.xlim(-6, 10)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("Z-Vertex Position (cm)", fontsize=25)
plt.ylabel("Counts $(x10^4)$", fontsize=25)
plt.legend(fontsize=25)
plt.show()
# ML_TS_pred.savefig("ML_TS_pred")

In [ ]:
#Butanol vs Carbon mmsp (CHECK)
g9a_mmsq_hist_pred = plt.figure(figsize=(15,12))

target_label = df_GMSH.T_pred.unique()
ax=plt.axes()

for i in target_label:
    hit = df_GMSH[df_GMSH.T_pred == i]
    if i == 0:
        target = "Butanol"
        n_buta, b_buta, p_buta = ax.hist(hit.mmsq_pi0, alpha=0.5, bins=1000, histtype='step', label=target, \
                                         color='#55A868', lw=3) 
        bin_max_buta = np.argmax(n_buta)
        buta_peak    = b_buta[bin_max_buta]
        #plt.axvline(x = b[bin_max_buta], linestyle="--", color = "g")
    elif i== 1:
        target = "Carbon"
        n_carb, b_carb, p_carb = ax.hist(hit.mmsq_pi0, alpha=0.5, bins=b_buta, histtype='step', label=target, \
                                         color='#C44E52', lw=3) 
        bin_max_carb = np.argmax(n_carb)
        carb_peak    = b_carb[bin_max_carb]
        #plt.axvline(x = b[bin_max_carb], linestyle="--", color = "r")
    elif i==2:
        target = "Polythene"
        
#Texts
plt.text(0.75, 0.8, " Butanol Peak {:>10.3f} \n Carbon Peak {:>11.3f} " \
         .format(buta_peak, carb_peak), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)        

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/10000)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

#lim, ticks, label, legends
# plt.xticks(np.arange(-1, 1, 0.1))
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25)
plt.xlim(-1, 1)
plt.xlabel("$m^2_{\pi_0}$ ($GeV^2$)", fontsize=30)
plt.ylabel("Counts $(x10^4)$", fontsize=30)

plt.show()
# g9a_mmsq_hist_pred.savefig("./run_period_1/g9a_mmsq_hist_pred")
# g9a_mmsq_hist_pred.savefig("./run_period_2/g9a_mmsq_hist_pred")
# g9a_mmsq_hist_pred.savefig("./run_period_3/g9a_mmsq_hist_pred")

#### Build Model + Prediction (LLapi)

###### Notes 

In [ ]:
#Target Prediction - on Same axes
ML_TS_pred   = plt.figure(figsize=(15,15))

#Plot
n_buta, b_buta, p_buta = plt.hist(df_GMSH[df_GMSH["T_pred"]==0]["z"], bins=1000, label="Butanol", alpha=0.7, color="#55A868")
n_carb, b_carb, p_carb = plt.hist(df_GMSH[df_GMSH["T_pred"]==1]["z"], bins=500, label="Carbon", alpha=0.7, color="#C44E52")

#Stats
peak_buta = b_buta[np.argmax(n_buta)]
mean_buta = df_GMSH[df_GMSH["T_pred"]==0]["z"].astype(float).mean()
std_buta  = df_GMSH[df_GMSH["T_pred"]==0]["z"].astype(float).std()
peak_carb = b_carb[np.argmax(n_carb)]
mean_carb = df_GMSH[df_GMSH["T_pred"]==1]["z"].astype(float).mean()
std_carb  = df_GMSH[df_GMSH["T_pred"]==1]["z"].astype(float).std()

#Texts
plt.text(0.05, 0.85, "Butanol \n Peak {:>10.3f} \n $\mu$ {:>15.3f} \n $\sigma$ {:>15.3f}" \
         .format(peak_buta, mean_buta, std_buta), \
         fontsize=30, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)
plt.text(0.05, 0.65, "Carbon \n Peak {:>10.3f} \n $\mu$ {:>15.3f} \n $\sigma$ {:>15.3f}" \
         .format(peak_carb, mean_carb, std_carb), \
         fontsize=30, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)


#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/10000)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))
        
plt.xlim(-6, 10)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlabel("Z-Vertex Position (cm)", fontsize=30)
plt.ylabel("Counts $(x10^4)$", fontsize=30)
plt.legend(fontsize=25)
plt.show()
# ML_TS_pred.savefig("ML_TS_pred")

In [1]:
# * Bias term = Fix the offset of the activation funciton to the left or to the right (NOTES)
#   $$ activation = sig(w_0*x + w_1*1.0) $$
#     * where x = input, bias = 1.0, w_0 = weight of input, w_1 = weight of bias

In [90]:
df_GMSH.columns

Index(['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'epho', 'x',
       'y', 'z', 'TRIGBITS', 'el_px', 'el_py', 'el_pz', 'el_E', 'timediff',
       'runNum', 'p_abs', 'el_p_abs', 'mmsq_pi0', 'el_mmsq_pi0', 'theta',
       'phi', 'sector', 'tof_pad', 'beta_diff', 'w', 'cthe_cm', 'tar_Pol',
       'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
       'pol_pho_stat', 'T_fin', 'el_pc_p_abs', 'el_pc_mmsq_pi0'],
      dtype='object')

###### Code 

In [ ]:
#Change data formats of train_label: 0, 1, 2 -> (0, 0, 1), (0, 1, 0), (1, 0, 0)
#------------------------------------------------------------------------------------------------------

def label_chng(truth):
    butanol   = 0
    carbon    = 0
    polythene = 0
    if   (truth == 0):
        butanol   = 1
    elif (truth == 1):
        carbon    = 1
    elif (truth == 2):
        polythene = 1
    return butanol, carbon, polythene
    
#Vectorize
vect_label_chng            = np.vectorize(label_chng, otypes=[np.int, np.int, np.int])
butanol, carbon, polythene = vect_label_chng(df_train_label["truth"].values) 

#Append to df
df_train_label["butanol"]    = pd.Series(butanol, index=df_train_label.index)
df_train_label["carbon"]     = pd.Series(carbon, index=df_train_label.index)
df_train_label["polythene"]  = pd.Series(polythene, index=df_train_label.index)

#Drop original "truth" column
df_train_label.drop(columns=["truth"], inplace=True)

#Convert to numpy array
train_label = df_train_label.as_matrix() 

In [ ]:
#Parameter Setting
#------------------------------------------------------------------------------------------------------

#learning rate: W(i) = W(i-1) - learning_rate * gradient
# learning_rate = 0.1
learning_rate = 0.05

#number of batches of data to train
# num_steps     = 5
num_steps     = 500

#batch size: Divide training data into batches of ~1000 events (no reason...)
# batch_size    = int(len(train_all) / 5)
batch_size    = int(len(train_all) / 500)

#Display loss scores at every display_step intervals
display_step  = 50


#14 columns for each event data
num_input   = 14
#3 classes of results
num_classes = 3

#number of neurons in 1st and 2nd layers
#    -VARY THIS AND SEE HOW IT CHANGES!!!!!!!!!!
n_hidden_1  = 14
n_hidden_2  = 3

#tf Graph input - X=input, Y=output, None means shape not fixed yet (# of rows not fixed)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
#Layer weight & bias
#------------------------------------------------------------------------------------------------------

weights = {"h1" : tf.Variable(tf.random_normal([num_input, n_hidden_1])), \
           "h2" : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])), \
           "out": tf.Variable(tf.random_normal([n_hidden_2, num_classes]))}

biases  = {"b1" : tf.Variable(tf.random_normal([n_hidden_1])), \
           "b2" : tf.Variable(tf.random_normal([n_hidden_2])), \
           "out": tf.Variable(tf.random_normal([num_classes]))}

In [ ]:
#Create Neural Network fn
#------------------------------------------------------------------------------------------------------

def neural_net(x):
    #Hidden fully connected layer with 14 neurons
    #    -multiple input (x) by weights ([num_input, n_hidden_1])
    #    -Add first biase["b1"] to center it
    layer_1   = tf.add(tf.matmul(x, weights["h1"]), biases["b1"])
    
    #Relu activation 
    relu_1    = tf.nn.relu(layer_1)
    
    #Hidden fully connected layer with 14 neurons
    layer_2   = tf.add(tf.matmul(relu_1, weights["h2"]), biases["b2"])    

    #Output fully connected layer with a neuron (node) for each class
    out_layer = tf.nn.softmax(tf.matmul(layer_2, weights["out"]) + biases["out"])
    
    #return final layer output in same dim as input
    return out_layer

In [ ]:
#Compile model - optimizer + loss fn
#------------------------------------------------------------------------------------------------------

#Construct model
#    -logits = unscaled log probabilityes -> results from output layer
logits          = neural_net(X)

#Loss function
#    -reduce_mean() computes mean across dimenstions of tensor
#    -logits        = Unscaled log probabilities
#    -cross_entropy = inv. proportional to probability - Computes cross entropy of `logits` and `labels`
#    -softmax       = returns array of prob. scores that sum to 1 - IOW, normalized
loss_op         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

#Optmizer
optimizer       = tf.train.AdagradOptimizer(learning_rate=learning_rate)

#Train Operator = optimizer to minimize loss function output
train_op        = optimizer.minimize(loss_op)

#Evaluate model
#    -with test logits, for dropout to be disabled
#    -tf.argmax(logits, 1) = highest value in logits (vector) values
#    -tf.argmax(Y, 1) = highest values in Y (output)
#    -tf.equal() = returns truth results 
#    -tf.cast() = casts a tensor to a new type, not altering dimesions of the tensor
correct_pred    = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1)) 
accuracy_op     = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
#Training
#------------------------------------------------------------------------------------------------------

#Initializa the variables (assign default values)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    #Run the initializer to initialize variables to defaults
    sess.run(init)
    
    #prediction_before = sess.run(logits, feed_dict={X:test_all})
    #print("logits on testdata: \n", prediction_before)

    for step in range(0, num_steps):
        #loop over batches of data
        #    -batch_x = (batch_size, num_input)
        #    -batch_y = (batch_size, num_classes)
        batch_x = []
        batch_y = []
        for i in range(0, batch_size):
            batch_x.append(train_all[i + step*batch_size])
            batch_y.append(train_label[i + step*batch_size])
        batch_x = np.asarray(batch_x)
        batch_y = np.asarray(batch_y)
        
        #Run optimization operation (backprop)
        #    -X goes into logits = neural_net(X), which is the starting input values
        #    -Y goes into loss_op, which are labels (truth values?)
        sess.run(train_op, feed_dict={X: batch_x, Y:batch_y})
        
        #Show status
        if (step % display_step == 0 or step == 0 or step ==num_steps-1):
            #Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy_op], feed_dict={X:batch_x, Y:batch_y}) 
            print("Step " + str(step) + \
                  ", \t\t Minibatch Loss = {:.4f}".format(loss) + \
                  ", \t\t Training Accuracy = {:.3f}".format(acc))
    print("Optimization Finished!") 

    #Calculate accuracy for MNIST test images
    #    -My test data does not have ground truth values. Just make the predictions and append
    #print("#-------------------------------------------------------------------------------------")
    #print("Testing Accuracy: ", sess.run(accuracy_op, feed_dict={X:test_all, Y:mnist.test_label}))
    
    
#Predictions on test data - has to be done within same session()
#------------------------------------------------------------------------------------------------------
    prediction = sess.run(logits, feed_dict={X:test_all})
    print("logits on testdata: \n", prediction)
    
    
    print(sess.run(weights["h1"]))  
    print(sess.run(weights["h1"]))

In [ ]:
#predictions_list is a list of final predictions of target on test_all data
predictions_list = []
for i in range(0, len(prediction)):
    predictions_list.append(np.argmax(prediction[i]))

In [ ]:
#Append prediction to df
df_GMSH["T_ll_pred"] = pd.Series(predictions_list, index=df_GMSH.index)

In [ ]:
#Target Prediction
g9a_hit_pos_yz_hist_pred_sep, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(15,15))

#target label
target_label = df_GMSH.T_ll_pred.unique()

#Grids
for i in [ax1, ax2, ax3]:
    i.axes.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    i.axes.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    i.grid(b=True, which="major", color="w", linewidth=1.0)
    i.grid(b=True, which="minor", color="w", linewidth=0.5)

for i in target_label:
    hit = df_GMSH[df_GMSH.T_ll_pred == i]
    if i == 0:
        target = "Butanol"
        ax1.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 
    elif i== 1:
        target = "Carbon"
        ax2.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 
    elif i==2:
        target = "Polythene"
        ax3.hist(hit.z, alpha=0.5, bins=500, histtype='bar', label=target) 
                        
ax1.set_title("Butanol")
ax2.set_title("Carbon")
ax3.set_title("Polythene")
ax1.set_xlim(-10, 20)
ax2.set_xlim(-10, 20)
ax3.set_xlim(-10, 20)
plt.xlabel("Z-Vertex Position (cm)")
# plt.xlim(-10,20)


plt.show()
# g9a_hit_pos_yz_hist_pred_sep.savefig("g9a_hit_pos_yz_hist_pred_sep")

### Hydrogen Contamination 

#### Training Data (MUST)

###### Notes for Ice training

* Possible training events for ICE (NOTES)
 1. Tight cut on the mmsq_pi0 sharp peak on the g9a Carbon data + events with z-vertex > 7cm.
     * `Since ice was reported to have been form on the right side of the Carbon target. [Steffen Strauch's Analysis page of FROST wiki]` 
     * Since bound-nucleon events with Fermi momentum will contribute the mmsq peak to be broad and the free-nucleon
      events will have sharper mmsq peak.
    * Selecting sharper peak reduces background (carbon) - __there is no perfectly pure free-nucleon training data__.
      1. Create a df from events that are labeled as Carbon fron the _Target Selection_ section.
      2. Make a mmsq_pi0 plot from these events, select events that fall in the sharp peak. Use them as ICE 
        * Calculate BIN WIDTH $\rightarrow$ sharp peak position $\pm$ BIN WIDTH / 2 $\rightarrow$ ice training data!
  2. MC simulated free-nucleon events that will correspond to the sharp peak on the Carbon or Butanol
  
  
* 2 possible training events for Carbon
  1. Use g9b Carbon data as the bound-nucleaon traianing events 
    * Only single-tracked PROTONS only! Otherwise, mmsq is too messed up and the NN model will fail
  2. Use the very negative portion of mmsq_pi0 as the bound-nucleon training events.
    * Very positive region of mmsq_pi0 is not good since there are multi-pion events where only the protons are
      detected.
    * Concerned that selecting only negative mmsq_pi0 would lead to biased training
      * Just don't use mmsq_pi0 as classifying parameter then!
      * Select out events that have very negative mmsq_pi0, see there distribution in yz hit positions
        * If they are dispersed well throughout the left side of the carbon, then we can use these events as
          training events for bound-nucleon events
          
* `Study training data size dependence predictions`
* `Study # of epochs dependence on predictions`

###### Notes for Carbon training

(NOTES)
* if peak for mmsq_pi0 is broader, they are from bound-nucleon events (carbon) due to Fermi momentum
* If the events' z-vertex is in [5.5, 6.5] , then they are certainly from carbon, since ice was reported to have been formed on the right side region of carbon target.

###### Code for Ice training (MUST)

In [56]:
#Predicted Carbon (MUST)
df_carbon_pred = df_GMSH[df_GMSH["T_pred"] == 1]

In [ ]:
#Carbon predicted (CHECK)
g9a_yz_carbon_pred = plt.figure(figsize=(15,15))
ax                 = plt.axes()
n, b, p = ax.hist(df_carbon_pred.z, bins=500, alpha=0.6, histtype='bar', label="carbon")
for i in range(0, int(len(p))):
    if b[i] >= 5.0 and b[i] <= 7.0:
        p[i].set_facecolor('r')

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="w", linewidth=1.0)
plt.grid(b=True, which="minor", color="w", linewidth=0.5)

plt.xlim(2, 10)
plt.axvline(x=7.0, linestyle = '--')
plt.axvline(x=5.0, linestyle = '--')
# plt.legend()
plt.xlabel("Z-Vertex Position (cm)")
plt.title("Predicted Carbon Events")
# plt.text(8.0, 100, "{:10}{:3}{:.4f} GeV\n{:20}{:3}{:.4f} GeV"\
#                     .format('Peak', " = ", 0.0245, "$\sigma$", " = ", df_carbon_pred.z.std())) 
plt.show()
# g9a_yz_carbon_pred.savefig("g9a_yz_carbon_pred")

bin_max = np.argmax(n)
print("b[bin_max]: ", b[bin_max])

In [ ]:
#Ice selection from Carbon data (MUST)
g9a_mmsq_pi0_carbon_pred = plt.figure(figsize=(15,15))
ax                       = plt.axes()
n, b, p = ax.hist(df_carbon_pred.mmsq_pi0, bins=1000, alpha=0.8, histtype='bar', color="#4C72B0")

#stats
bin_max     = np.argmax(n)
ice_up_lim  = b[bin_max] + df_carbon_pred["mmsq_pi0"].astype(float).std()/10
ice_low_lim = b[bin_max] - df_carbon_pred["mmsq_pi0"].astype(float).std()/10

#Color specific patches
for i in range(0, int(len(p))):
    if b[i] >= ice_low_lim and b[i] <= ice_up_lim:
        p[i].set_facecolor('#C44E52')
        
#Custom Legend        
custom_label = [mpl.patches.Patch(facecolor="#C44E52", alpha=0.8, label="ice train"), \
                mpl.patches.Patch(facecolor="#4C72B0", alpha=0.8, label="carbon")]
plt.legend(handles=custom_label, fontsize=30)

#axvlines
plt.axvline(ice_low_lim, linestyle="--")
plt.axvline(ice_up_lim, linestyle="--")    
        
#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/100)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

#ranges
plt.xlim(-1, 1)

#labels, ticks
plt.xlabel("$m^2_{\pi_0}$ ($GeV^2$)", fontsize=30)
plt.ylabel("Counts $(x10^2)$", fontsize=30)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)

#Texts
plt.text(0.75, 0.78, " Peak {:>10.3f} \n $\sigma$ {:>15.3f}".format(b[bin_max], df_carbon_pred.astype(float).mmsq_pi0.std()), \
         fontsize=30, transform=plt.gca().transAxes, bbox=dict(facecolor='none', pad=5.0)) 


plt.show()
# g9a_mmsq_pi0_carbon_pred.savefig("g9a_mmsq_pi0_carbon_pred")

print("Mode: ", b[bin_max])

In [59]:
#Ice train 1 (MUST)
df_ice_train = df_carbon_pred[(df_carbon_pred["mmsq_pi0"] >= ice_low_lim) & \
                              (df_carbon_pred["mmsq_pi0"] <= ice_up_lim)  & \
                              ((df_carbon_pred["z"] >= 6.) & (df_carbon_pred["z"] <= 7.5))]

In [ ]:
#Evidence of ice from Stenffen (CHECK)
Image(filename="ice_evidence_Steffen_2.png")

###### Code  for Carbon training (MUST)

In [ ]:
#hitpos_x_maxbin, hitpos_y_maxbin (MUST)
hitpos_x_maxbin = plt.figure(figsize=(12, 6))
n, b_x, p = plt.hist(df_GMSH["x"], bins=200, range=[-1.5, 1.5])

#stats
n_max_x = np.argmax(n)
plt.axvline(b_x[n_max_x], linestyle="--", color="#8C0900")
plt.text(0.8, 0.8, "Max bin = {:.3f}".format(b_x[n_max_x]), fontsize=15, bbox={"facecolor":"none", "pad":5.0}, \
         transform=plt.gca().transAxes)


plt.xlabel("x (cm)", fontsize=20)
plt.show()
#==========================================================================================================================

hitpos_y_maxbin = plt.figure(figsize=(12, 6))
n, b_y, p = plt.hist(df_GMSH["y"], bins=200, range=[-1.5, 1.5])

#stats
n_max_y = np.argmax(n)
plt.axvline(b_y[n_max_y], linestyle="--", color="#8C0900")
plt.text(0.8, 0.8, "Max bin = {:.3f}".format(b_y[n_max_y]), fontsize=15, bbox={"facecolor":"none", "pad":5.0}, \
         transform=plt.gca().transAxes)

plt.xlabel("y (cm)", fontsize=20)
plt.show()

x_center = round(b_x[n_max_x],4)
y_center = round(b_y[n_max_y],4)

print("x_center = ", x_center)
print("y_center = ", y_center)

In [531]:
#Carbon Training Data (MUST)
#    -x_center, y_center computed in He-bath selection 

df_carbon_train = df_carbon_pred[((df_carbon_pred["mmsq_pi0"] <= ice_low_lim)  | \
                                 (df_carbon_pred["mmsq_pi0"] >= ice_up_lim))   & \
                                 ((df_carbon_pred["z"] <= 6.76) & (df_carbon_pred["z"] >= 5.54)) & \
                                 ((pow(df_carbon_pred["x"] - (x_center), 2) + pow(df_carbon_pred["y"] - (y_center), 2)) < 0.8**2)]


In [ ]:
#Carbon training (CHECK)
carbon_training, ax = plt.subplots(2, 1, figsize=(15, 15))

ax[0].hist(df_carbon_train["mmsq_pi0"], bins=500, range=[-1, 1])
ax[1].hist(df_carbon_train["z"], bins=512, range=(5, 7))

#grids
for i in ax:
    i.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    i.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    i.grid(b=True, which="major", color="w", linewidth=1.0)
    i.grid(b=True, which="minor", color="w", linewidth=0.4)
    
#ticks change fontsize
for i in ax:
    i.tick_params(labelsize=15) 

ax[0].set_xlabel("$m^2_{\pi_0}$ ($GeV^2$)", fontsize=30)    
ax[0].set_ylabel("Counts", fontsize=30)    
ax[1].set_xlabel("$z-vertex$ ($cm$)", fontsize=30)    
ax[1].set_ylabel("Counts", fontsize=30)    
    
plt.show()
# carbon_training.savefig("carbon_train_hydrocon")

In [ ]:
#FROST target layout (CHECK)
Image(filename="frost_target_position.png")

###### Finding adequate size of training dataset for Ice and Carbon (MUST)

In [532]:
num_ice_train  = 2500
num_carb_train = int((len(df_carbon_train) / len(df_ice_train)) * num_ice_train)*3

print("# of total ICE training events (before random selection)    = ", len(df_ice_train))
print("# of total CARBON training events (before random selection) = ", len(df_carbon_train))

print("# of ICE training events (value chosen)          = ", num_ice_train)
print("# of carbon training events to randomly select   = ", num_carb_train)

# of total ICE training events (before random selection)    =  3672
# of total CARBON training events (before random selection) =  101434
# of ICE training events (value chosen)          =  2500
# of carbon training events to randomly select   =  207177


In [ ]:
#Select Random points for reduced training data
df_ice_train   = df_ice_train.sample(num_ice_train)

#Select Random points for reduced training data
df_carbon_train   = df_carbon_train.sample(num_carb_train)

In [534]:
#When training data too low

#Select Random points for reduced training data
df_ice_train   = df_ice_train.sample(num_ice_train, replace=True)

#Select Random points for reduced training data
df_carbon_train   = df_carbon_train.sample(num_carb_train, replace=True)

###### Training Data Finalize (MUST)

In [535]:
#Ground Truth column - train labels
df_ice_train["truth"]    = 0
df_carbon_train["truth"] = 1

In [536]:
#Columns list
df_ice_train_col    = df_ice_train.columns.tolist()

#Align the column order
df_carbon_train     = df_carbon_train[df_ice_train_col]
df_carbon_train_col = df_carbon_train.columns.tolist()

In [537]:
#Combine trainig data of ice + carbom
df_train_all = pd.concat([df_ice_train, df_carbon_train])

#Separate df for ground truth - train label
df_train_label = pd.DataFrame(df_train_all["truth"])

#Drop groun truth column in training data
df_train_all.drop(columns=["truth"], inplace=True)

In [538]:
df_train_all.columns

Index(['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'epho', 'x',
       'y', 'z', 'scPdHt', 'TRIGBITS', 'el_px', 'el_py', 'el_pz', 'el_E',
       'timediff', 'runNum', 'p_abs', 'el_p_abs', 'mmsq_pi0', 'el_mmsq_pi0',
       'theta', 'phi', 'sector', 'tof_pad', 'beta_diff', 'phi_sec',
       'el_pc_p_abs', 'el_pc_pz', 'el_pc_mmsq_pi0', 'w', 'beta_cm', 'gamma_cm',
       'pz_cm', 'p_cm_abs', 'theta_cm', 'cthe_cm', 'tar_Pol', 'tar_Pol_stat',
       'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho', 'pol_pho_stat',
       'T_pred'],
      dtype='object')

In [539]:
#drop the columns that df_train_all dropped already

df_train_all = df_train_all.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'scPdHt', 'TRIGBITS',
                                           'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
                                           'sector', 'tof_pad', 'beta_diff', 'w', 'beta_cm',
                                           'gamma_cm', 'pz_cm', 'p_cm_abs', 'theta_cm', 'cthe_cm', 'tar_Pol',
                                           'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
                                           'pol_pho_stat', "T_pred"])

In [540]:
# #Drop when doing 2nd time
# df_train_all.drop(columns=["ice_pred", "T_fin"], inplace=True)

In [541]:
#Convert to numpy ndarray 
train_all   = df_train_all.values
train_label = df_train_label.values

In [542]:
print(len(train_all))
print(len(train_label))
print(train_all.shape)
print(train_label.shape)

209677
209677
(209677, 16)
(209677, 1)


#### Test Data (MUST)

In [543]:
#drop the columns that df_train_all dropped already
df_GMSH_ml = df_GMSH.drop(columns=['pid', 'E', 'px', 'py', 'pz', 'betac', 'mass', 'betam', 'scPdHt', 'TRIGBITS',
                                   'timediff', 'runNum', 'p_abs', 'mmsq_pi0',
                                   'sector', 'tof_pad', 'beta_diff', 'w', 'beta_cm',
                                   'gamma_cm', 'pz_cm', 'p_cm_abs', 'theta_cm', 'cthe_cm', 'tar_Pol',
                                   'tar_Pol_stat', 'tar_Pol_sys', 'tar_Pol_sign', 'bm_helicity', 'pol_pho',
                                   'pol_pho_stat', "T_pred"])

In [544]:
# #Drop when doing 2nd time
# df_train_all.drop(columns=["ice_pred", "T_fin"], inplace=True)

In [545]:
#Change order of columns so that columns of training data and test data matches¶
df_train_all_cols = df_train_all.columns.tolist()
df_GMSH_ml = df_GMSH_ml[df_train_all_cols]

print(df_train_all_cols)
print(df_GMSH_ml.columns.tolist())

['epho', 'x', 'y', 'z', 'el_px', 'el_py', 'el_pz', 'el_E', 'el_p_abs', 'el_mmsq_pi0', 'theta', 'phi', 'phi_sec', 'el_pc_p_abs', 'el_pc_pz', 'el_pc_mmsq_pi0']
['epho', 'x', 'y', 'z', 'el_px', 'el_py', 'el_pz', 'el_E', 'el_p_abs', 'el_mmsq_pi0', 'theta', 'phi', 'phi_sec', 'el_pc_p_abs', 'el_pc_pz', 'el_pc_mmsq_pi0']


In [546]:
#convert df_GPID_MVRT to numpy ndarry
test_all = df_GMSH_ml.values

#### Summary of training + test data 

So far we have the following data sets:
* test_all = test data set 
  * df_GMSH_ml
* train_all = training data set
  * df_train_all 
* train_label  = trainig label (true values)
  * df_train_label

#### Build Model + Prediction (keras) (MUST)

In [547]:
#Layer Setup
model_ice = keras.Sequential()
model_ice.add(keras.layers.Dense(15, activation=tf.nn.relu))
model_ice.add(keras.layers.Dense(15, activation=tf.nn.relu))
# model_ice.add(keras.layers.Dense(15, activation=tf.nn.relu))
# model_ice.add(keras.layers.Dense(15, activation=tf.nn.relu))
model_ice.add(keras.layers.Dense(2, activation=tf.nn.softmax))

In [548]:
#Compile: Optimizer + Loss function
model_ice.compile(optimizer  = tf.optimizers.Adam(),
                    loss     = 'sparse_categorical_crossentropy',
                    metrics  = ['accuracy'])

In [ ]:
#Training
model_ice.fit(train_all, train_label, epochs=3)

In [550]:
#Make predictions
predictions_ice = model_ice.predict(test_all)

In [551]:
#predictions_ice
predictions_list = []
for i in range(0, len(predictions_ice)):
    predictions_list.append(np.argmax(predictions_ice[i]))

In [552]:
#Append predicted values (predictions_list) to df
df_GMSH["ice_pred"] = pd.Series(predictions_list, index=df_GMSH.index)

In [553]:
#Target selection final output into df_GHSM (MUST)
#    -Oprimized Version - np.vertorize()
def T_fin(T_pred, ice_pred):
    tfin = 0
    #Butanol
    if  (T_pred == 0):
        tfin = 0
    #Ice
    elif(T_pred == 1 and ice_pred == 0):
        tfin = 3
    #Carbon
    elif(T_pred == 1 and ice_pred == 1):
        tfin = 1
    #Polythene
    elif(T_pred == 2):
        tfin = 2
    return tfin
    
#Vectorize
vect_T_fin             = np.vectorize(T_fin, otypes=[np.int])
T_fin                  = vect_T_fin(df_GMSH["T_pred"].values, df_GMSH["ice_pred"].values)

#Append to df
df_GMSH["T_fin"] = pd.Series(T_fin, index=df_GMSH.index)

In [ ]:
#final predictions plot (CHECK)
g9a_final_target_pred_all = plt.figure(figsize=(12,12))
target_label = df_GMSH.T_fin.unique()
ax=plt.axes()

for i in target_label:
    hit = df_GMSH[df_GMSH.T_fin == i]
    if   i == 0:
        target = "butanol"
        n_buta, b_buta, p_buta = ax.hist(hit.z, alpha=0.5, bins=600, label=target) 
    elif i== 1:
        target = "Carbon"
        n_carb, b_carb, p_carb = ax.hist(hit.z, alpha=0.5, bins=500, label=target) 
    elif i== 2:
        target = "polythene"
        n_poly, b_poly, p_poly = ax.hist(hit.z, alpha=0.5, bins=500, label=target) 
    elif i== 3:
        target = "ice"
        n_ice, b_ice, p_ice = ax.hist(hit.z, alpha=0.5, bins=500, label=target) 

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="w", linewidth=1.0)
plt.grid(b=True, which="minor", color="w", linewidth=0.5)
        
plt.xlim(-5,20)
plt.xlabel("Z-Vertex Position (cm)", fontsize=15)
plt.legend(fontsize=15)
# plt.title("", fontsize=15)
plt.show()
# g9a_final_target_pred_all.savefig("./run_period_2/g9a_final_target_pred_all")
# g9a_final_target_pred_all.savefig("./run_period_3/g9a_final_target_pred_all")

In [ ]:
#Final predictions Z-Vertex - only for carbon and ice (CHECK)
g9a_final_target_pred_ca_ice = plt.figure(figsize=(12,12))
target_label = df_GMSH.T_fin.unique()
ax=plt.axes()

for i in target_label:
    hit = df_GMSH[df_GMSH.T_fin == i]
#     if   i == 0:
#         target = "butanol"
#         ax.hist(hit.z, alpha=0.5, bins=1000, label=target) 
    if i== 1:
        target = "carbon"
        n_carb, b_carb, p_carb = ax.hist(hit.z, alpha=0.5, bins=300, label=target) 
#     elif i== 2:
#         target = "polythene"
#         ax.hist(hit.z, alpha=0.5, bins=1000, label=target) 
    elif i== 3:
        target = "ice"
        n_ice, b_ice, p_ice = ax.hist(hit.z, alpha=0.5, bins=b_carb, label=target) 

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="grey", linewidth=1.0)
plt.grid(b=True, which="minor", color="grey", linewidth=0.5)
        
plt.xlim(2,12)
plt.xlabel("Z-Vertex Position (cm)", fontsize=15)
plt.legend(fontsize=15)
# plt.title("", fontsize=15)
plt.show()
# g9a_final_target_pred_ca_ice.savefig("./run_period_2/g9a_final_target_pred_ca_ice")
# g9a_final_target_pred_ca_ice.savefig("./run_period_3/g9a_final_target_pred_ca_ice")

In [ ]:
#Final predictions MMSQ -  only for carbon and ice (CHECK)
g9a_final_target_pred_ca_ice_mmsq = plt.figure(figsize=(12,12))
target_label = df_GMSH.T_fin.unique()
ax=plt.axes()

for i in target_label:
    hit = df_GMSH[df_GMSH.T_fin == i]
#     if   i == 0:
#         target = "butanol"
#         ax.hist(hit.z, alpha=0.5, bins=1000, label=target) 
    if i== 1:
        target = "carbon"
        ax.hist(hit.mmsq_pi0, alpha=0.5, bins=500, label=target, color="#4C72B0") 
#     elif i== 2:
#         target = "polythene"
#         ax.hist(hit.z, alpha=0.5, bins=1000, label=target) 
    elif i== 3:
        target = "ice"
        ax.hist(hit.mmsq_pi0, alpha=0.5, bins=500, label=target, color="#C44E52") 

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="w", linewidth=1.0)
plt.grid(b=True, which="minor", color="w", linewidth=0.5)
        
plt.xlim(-2,1)
plt.xlabel("$m^2_{\pi_0}$ ($GeV^2$)", fontsize=15)
plt.legend(fontsize=15)
# plt.title("", fontsize=15)
plt.show()
# g9a_final_target_pred_ca_ice_mmsq.savefig("./run_period_2/g9a_final_target_pred_ca_ice_mmsq")
# g9a_final_target_pred_ca_ice_mmsq.savefig("./run_period_3/g9a_final_target_pred_ca_ice_mmsq")

In [ ]:
#g9a_final_target_pred_ca_ice_mmsq_zvert (CHECK)
g9a_final_target_pred_ca_ice_mmsq_zvert, subplots = plt.subplots(3, 1, figsize=(20, 20)) 

a = subplots[0].hist2d(df_GMSH["z"][(df_GMSH["T_fin"] == 1) | (df_GMSH["T_fin"] == 3)], 
                       df_GMSH["mmsq_pi0"][(df_GMSH["T_fin"] == 1) | (df_GMSH["T_fin"] == 3)], 
                       cmap="jet", cmin=0, bins=300, range=[[4, 8], [-1.0, 1.0]], norm=mpl.colors.LogNorm())

b = subplots[1].hist2d(df_GMSH["z"][(df_GMSH["T_fin"] == 1)], df_GMSH["mmsq_pi0"][(df_GMSH["T_fin"] == 1)],
                       cmap="jet", cmin=0, bins=300, range=[[4, 8], [-1.0, 1.0]], norm=mpl.colors.LogNorm())

c = subplots[2].hist2d(df_GMSH["z"][(df_GMSH["T_fin"] == 3)], df_GMSH["mmsq_pi0"][(df_GMSH["T_fin"] == 3)],
                       cmap="jet", cmin=0, bins=300, range=[[4, 8], [-1.0, 1.0]], norm=mpl.colors.LogNorm())


#Add axes
cbaxes_0 = g9a_final_target_pred_ca_ice_mmsq_zvert.add_axes([0.91, 0.125, 0.03, 0.756])   # x_pos, y_pos, width, length

#Set Colorbar ([3] indicated the subplot)
cbar = g9a_final_target_pred_ca_ice_mmsq_zvert.colorbar(a[3], cax=cbaxes_0)

#Set Colorbar limit
# a[3].set_clim(0, 100)
# b[3].set_clim(0, 100)
# c[3].set_clim(0, 100)

#Change Color Bar fontsize
cbar.ax.tick_params(labelsize=20)

#X and Y labels
# subplots[0].set_xlabel("Z-Vertex (cm)", fontsize=25)
# subplots[1].set_xlabel("Z-Vertex (cm)", fontsize=25)
subplots[2].set_xlabel("Z-Vertex (cm)", fontsize=40)
subplots[0].set_ylabel("$m^2_{\pi_0}$ $(GeV^2)$", fontsize=30)
subplots[1].set_ylabel("$m^2_{\pi_0}$ $(GeV^2)$", fontsize=30)
subplots[2].set_ylabel("$m^2_{\pi_0}$ $(GeV^2)$", fontsize=30)

#Ticks
subplots[0].tick_params(labelsize=25)
subplots[1].tick_params(labelsize=25)
subplots[2].tick_params(labelsize=25)

#Layout change
# age_IPSYN_all.tight_layout()

#Title
subplots[0].set_title("Carbon + Ice", fontsize=35)
subplots[1].set_title("Carbon", fontsize=35)
subplots[2].set_title("Ice", fontsize=35)

#Spacing
g9a_final_target_pred_ca_ice_mmsq_zvert.subplots_adjust(hspace=0.35)

plt.show()
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_1//g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_2/g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_3//g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_4_1//g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_5_1//g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_6_1//g9a_final_target_pred_ca_ice_mmsq_zvert")
# g9a_final_target_pred_ca_ice_mmsq_zvert.savefig("./run_period_7_1//g9a_final_target_pred_ca_ice_mmsq_zvert")

In [ ]:
#Butanol vs Carbon - ice mmsp (CHECK)
g9a_mmsq_pred_ml = plt.figure(figsize=(15,15))

#Plot
n_buta, b_buta, p_buta = plt.hist(df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==0], alpha=1, bins="fd", histtype='step', label="butanol", linewidth=2) 
bin_max_buta = np.argmax(n_buta)
n, b, p = plt.hist(df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==1], alpha=1, bins=b_buta, histtype='step', label="carbon", linewidth=2) 
bin_max_carb = np.argmax(n)
# n, b, p = plt.hist(df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==2], alpha=1, bins=b_buta, histtype='step', label="polythene", linewidth=2) 
# bin_max_poly = np.argmax(n)
n, b, p = plt.hist(df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==3], alpha=1, bins=b_buta, histtype='step', label="ice", linewidth=2)
bin_max_ice = np.argmax(n)


#Texts
plt.text(0.05, 0.85, "BUTANOL \n Mode {:>10.3f} \n Mean {:>10.3f} \n $\sigma$ {:>15.3f}" \
         .format(b[bin_max_buta], df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==0].astype(float).mean(), \
                 df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==0].astype(float).std()), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

plt.text(0.05, 0.73, "CARBON \n Mode {:>10.3f} \n Mean {:>10.3f} \n $\sigma$ {:>15.3f}" \
         .format(b[bin_max_carb], df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==1].astype(float).mean(), \
                 df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==1].astype(float).std()), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

# plt.text(0.05, 0.61, "POLYTHENE \n Mode {:>10.3f} \n Mean {:>10.3f} \n $\sigma$ {:>15.3f}" \
#          .format(b[bin_max_poly], df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==2].astype(float).mean(), \
#                  df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==2].astype(float).std()), \
#          fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

plt.text(0.05, 0.49, "ICE \n Mode {:>10.3f} \n Mean {:>10.3f} \n $\sigma$ {:>15.3f}" \
         .format(b[bin_max_ice], df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==3].astype(float).mean(), \
                 df_GMSH["mmsq_pi0"][df_GMSH["T_fin"]==3].astype(float).std()), \
         fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

#Labels and lim
plt.xlim(-1, 1)
plt.xlabel("Missing mass sq. $[GeV^2/c^4]$", fontsize=30)
plt.ylabel("Counts $(x10^4)$", fontsize=30)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/10000)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

# plt.title("Predicted Missing Mass Sq $\pi_0$: Butanol vs Carbon")
plt.legend(fontsize=20)
plt.show()

# g9a_mmsq_pred_ml.savefig("./run_period_1/g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_2/g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_3/g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_4_1//g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_5_1//g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_6_1//g9a_mmsq_pred_ml")
# g9a_mmsq_pred_ml.savefig("./run_period_7_1//g9a_mmsq_pred_ml")

#### Build Model + Prediction (LLapi) 

# Save and Resume 

### Data Save

###### Low Energy Run Range  1

In [97]:
#Save SUPER RAW (ELOSS + CM COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [80]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [10]:
# Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [164]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_1/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [70]:
#Read Save SUPER RAW (ELOSS + CM COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [21]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [31]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [10]:
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range  2 

In [196]:
# #Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [229]:
# #Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [186]:
# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
# #Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_2/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [43]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [39]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [214]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range  3

In [22]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [66]:
#Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [289]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_3/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [26]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [217]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [212]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range 4 - [0, 1.6] 

In [87]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [122]:
#Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [351]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_4/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [53]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [290]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [16]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range 5 - [0, 1.6] 

In [174]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [185]:
#Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [409]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_5/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [53]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [352]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [16]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range 6 - [0, 1.6] 

In [215]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [248]:
# #Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [74]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_6/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [66]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [18]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [16]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Low Energy Run Range 7 - [0, 1.6] 

In [280]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [312]:
# #Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [134]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Save All (mmsq)
# df_GMSH.to_pickle("./dstmakedstmaker_Ppi0_4_elossr_Ppi0_4/low_7/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [75]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read All (mmsq)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_ALL.pkl")

In [ ]:
df_GMSH

###### Run Range 4 - High Energy - (1.6, 2.4] 

In [40]:
# ##Save SUPER RAW (ELOSS + CM COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [134]:
# ##Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [317]:
# ##Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [559]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
# #Save after mmsq selection: mmsq_dict
# mmsq = open("./dstmaker_Ppi0_4_eloss/high_4/mmsq_dict.pkl", "wb")
# pickle.dump(mmsq_dict, mmsq)
# mmsq.close()

In [120]:
#Read Save SUPER RAW (ELOSS + CM COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [3]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [336]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [556]:
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read MMSQ selection: mmsq_dict
mmsq       = open("./dstmaker_Ppi0_4_eloss/high_4/mmsq_dict.pkl", "rb")
mmsq_dict  = pickle.load(mmsq)
mmsq.close()

In [ ]:
df_GMSH

###### Run Range 5 - High Energy - (1.6, 2.4] 

In [66]:
# ##Save SUPER RAW (ELOSS + CM COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [261]:
#Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [323]:
## Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [562]:
##Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
# #Save after mmsq selection: mmsq_dict
# mmsq = open("./dstmaker_Ppi0_4_eloss/high_5/mmsq_dict.pkl", "wb")
# pickle.dump(mmsq_dict, mmsq)
# mmsq.close()

In [254]:
#Read Save SUPER RAW (ELOSS + CM COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [318]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [5]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [568]:
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read MMSQ selection: mmsq_dict
mmsq       = open("./dstmaker_Ppi0_4_eloss/high_5/mmsq_dict.pkl", "rb")
mmsq_dict  = pickle.load(mmsq)
mmsq.close()

In [ ]:
df_GMSH

###### Run Range 6 - High Energy - (1.6, 2.4] 

In [66]:
# ##Save SUPER RAW (ELOSS + CM COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [282]:
# ##Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [329]:
# #Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [567]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
# #Save after mmsq selection: mmsq_dict
# mmsq = open("./dstmaker_Ppi0_4_eloss/high_6/mmsq_dict.pkl", "wb")
# pickle.dump(mmsq_dict, mmsq)
# mmsq.close()

In [275]:
#Read Save SUPER RAW (ELOSS + CM COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [7]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [8]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [563]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read MMSQ selection: mmsq_dict
mmsq       = open("./dstmaker_Ppi0_4_eloss/high_6/mmsq_dict.pkl", "rb")
mmsq_dict  = pickle.load(mmsq)
mmsq.close()

In [ ]:
df_GMSH

###### Run Range 7 - High Energy - (1.6, 2.4] 

In [119]:
# #Save SUPER RAW (ELOSS + CM COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [301]:
# #Save RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [335]:
# Save Initial (LowMome + proton + photon + tof) - FOR ML
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [573]:
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
# #Save after mmsq selection: mmsq_dict
# mmsq = open("./dstmaker_Ppi0_4_eloss/high_7/mmsq_dict.pkl", "wb")
# pickle.dump(mmsq_dict, mmsq)
# mmsq.close()

In [294]:
#Read Save SUPER RAW (ELOSS + CM COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_super_raw.pkl")

In [330]:
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

In [10]:
#Read Initial (LowMome + proton + photon + tof) - FOR ML
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial.pkl")

In [570]:
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read MMSQ selection: mmsq_dict
mmsq       = open("./dstmaker_Ppi0_4_eloss/high_7/mmsq_dict.pkl", "rb")
mmsq_dict  = pickle.load(mmsq)
mmsq.close()

In [ ]:
df_GMSH

### Data Save/Resume for all runes COMBINED

###### For SF + DF 

In [ ]:
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH_1= pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_2 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_3 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_4 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_5 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_6 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_7 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

In [ ]:
#Combine
df_GMSH = pd.concat([df_GMSH_1, df_GMSH_2, df_GMSH_3, df_GMSH_4, df_GMSH_5, df_GMSH_6, df_GMSH_7], \
                     ignore_index=True)

In [ ]:
#Drop unneeded columns to speed up computation
column_drop_list = []
columns          = df_GMSH.columns
needed_columns   = ["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]
drop_list        = []

for i in columns:
    if (i not in needed_columns):
        drop_list.append(i)
        
#Drop-
df_GMSH.drop(columns=drop_list, axis=1, inplace=True)

### Binning

###### Low Energy 

In [ ]:
#Read df from each runs (MUST, BUT NOT YET)
df_GMSH = pd.DataFrame()

for i in range(1, 8):
    df_GMSH_i = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_{:d}/df_GMSH_dstmaker_Ppi0_4_initial.pkl".format(i))[["epho", "cthe_cm", "el_pc_mmsq_pi0"]]
    df_GMSH = df_GMSH.append(df_GMSH_i, ignore_index=True)

df_GMSH = df_GMSH.reset_index(drop=True)

In [ ]:
#Plot all events' mmsq (MUST)
global_mmsq = plt.figure(figsize=(15, 10))

#Histogram
n, b, p = plt.hist(df_GMSH["el_pc_mmsq_pi0"], bins=6500, label="$M_X^2$", alpha=0.7, color="#4C72B0")

#Stats
bin_max = np.argmax(n)
peak    = b[bin_max]
mean    = df_GMSH["el_pc_mmsq_pi0"].mean()
stddev  = df_GMSH["el_pc_mmsq_pi0"].std()

#===================================================================================================
#Selection range
#    -SAVE to be used for dropping events
frst_ln = peak - 0.5*stddev
scnd_ln  = peak + 0.5*stddev
#===================================================================================================

#Draw lines
plt.axvline(first_ln, linestyle="-", linewidth=3, alpha=0.5, color="#C44E52")
plt.axvline(scnd_ln,  linestyle="--", linewidth=3, alpha=0.5, color="#C44E52")

#Texts
plt.text(0.05, 0.85, " {:<12} {:>10.3f} \n {:<12} {:>10.3f} \n {:<21} {:>10.3f}" \
         .format("peak", peak, "mean", mean, "$\sigma$", stddev), \
         fontsize=15, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

#Y-Axis Scaling
def adjust_y_axis(x, pos):
    return "{:.0f}".format(x/10000)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

#Labels, ticks, lim
plt.xlim(-1.0, 1.0)
plt.xlabel("$M_X^2$ $[GeV^2/c^4]$", fontsize=15)
plt.ylabel("Counts $(x10^4)$", fontsize=15)

#Legends
custom_label = [mpl.lines.Line2D([0], [0], color="#C44E52", alpha=0.7, linewidth=3, \
                                 label="peak - $\sigma$    = {:.3f}".format(frst_ln), linestyle="-"), 
                mpl.lines.Line2D([0], [0], color="#C44E52", alpha=0.7, linewidth=3, \
                                 label="peak + $\sigma$   = {:.3f}".format(scnd_ln), linestyle="--")]
plt.legend(handles=custom_label, fontsize=15, loc="upper right")

# plt.legend()
plt.show()
global_mmsq.savefig("./binning_global_mmsq")


In [ ]:
#Drop events outside of +/- sigma (MUST)
df_GMSH.drop(df_GMSH[(df_GMSH["el_pc_mmsq_pi0"] < frst_ln) | (df_GMSH["el_pc_mmsq_pi0"] > scnd_ln)].index, \
             inplace=True)

In [ ]:
#Binning Method 1 (MUST)
bin_partition_eg_perc = [i/20 for i in range(0, 21, 1)]
bin_pos_eg_list    = df_GMSH["epho"].quantile(bin_partition_eg_perc).tolist()
epho_bins          = [round(i, 4) for i in bin_pos_eg_list]

#cos_bins are dict now
cos_bins = {}
bin_partition_cos_perc = [i/10 for i in range(0, 11, 1)]

#Partitions
for i in range(len(epho_bins)-1):
    #cos binning per epho bin
    bin_pos_cos_list = df_GMSH["cthe_cm"][(epho_bins[i] <= df_GMSH["epho"]) & \
                                          (df_GMSH["epho"] <= epho_bins[i+1])] \
                                         .quantile(bin_partition_cos_perc).tolist()

    #Into cos_bins dict
    cos_bins["Eg_{:07.3f}".format(epho_bins[i])] = [round(i, 4) for i in bin_pos_cos_list]

#Print
print(bin_partition_eg_perc)
print(len(bin_partition_eg_perc))
print(bin_partition_cos_perc)
print(len(bin_partition_cos_perc))
print("epho_bins = ", epho_bins)
print("len(epho_bins) = ", len(epho_bins))
#Print cos bins for each epho bins
for i in sorted(cos_bins.keys()):
    print(i, cos_bins[i])

In [ ]:
#epho vs angle (CHECK)
binning = plt.figure(figsize=(15, 15))

#Hist
plt.hist2d(df_GMSH["cthe_cm"], df_GMSH["epho"], bins=500, cmap="jet", cmin=0, \
           norm=mpl.colors.LogNorm())

#Lines to indicate epho bins
for i, k in zip(sorted(cos_bins.keys()), range(len(epho_bins))):
    for j in range(len(cos_bins[i])):
        plt.hlines(y=epho_bins[k], xmin=cos_bins[i][0], xmax=cos_bins[i][-1], \
                   linestyle="-", linewidth=1.75, color="black")
        plt.hlines(y=epho_bins[k+1], xmin=cos_bins[i][0], xmax=cos_bins[i][-1], \
                   linestyle="-", linewidth=1.75, color="black")
        
#Manually draw last epho line
plt.hlines(y=epho_bins[-1], xmin=cos_bins[sorted(cos_bins.keys())[-1]][0], \
           xmax=cos_bins[sorted(cos_bins.keys())[-1]][-1], \
           linestyle="-", linewidth=1.75, color="black")
        
        
#Lines to indicate cos bins    
for i, k in zip(sorted(cos_bins.keys()), range(len(epho_bins)-1)):
    for j in range(len(cos_bins[i])):
        plt.vlines(x=cos_bins[i][j], ymin=epho_bins[k], ymax=epho_bins[k+1], \
                   linestyle="-", linewidth=1.75, color="black")
    
#lim, label, ticks    
plt.xlim(-1.1, 1.1)
plt.ylim(0.25, 1.75)
plt.xlabel("$cos\\theta_{cm}$", fontsize=40)
plt.ylabel("$E_{\gamma}$", fontsize=40)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#Grids
plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

cbaxes = binning.add_axes([0.91, 0.125, 0.02, 0.755])   # x_pos, y_pos, width, length
cbaxes.tick_params(labelsize=20)
plt.colorbar(cax=cbaxes)
plt.show
binning.savefig("./binning")
# binning.savefig("./binning_1")

In [ ]:
# #Binning method 1 # of events in each bins (CHECK)
# for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
#     print("==============================", epho_bins[i], "=============================")
#     for k in range(len(cos_bins[j])-1):

#         print(len(df_GMSH[(epho_bins[i]   <= df_GMSH["epho"])    & (df_GMSH["epho"]    <= epho_bins[i+1]) & \
#                           (cos_bins[j][k] <= df_GMSH["cthe_cm"]) & (df_GMSH["cthe_cm"] <= cos_bins[j][k+1])]))
        

In [ ]:
# #Binning Method 2 (NONEED)
# bin_partition_eg_perc = [i/20 for i in range(0, 21, 1)]
# bin_pos_eg_list    = df_GMSH["epho"].quantile(bin_partition_eg_perc).tolist()
# epho_bins          = [round(i, 4) for i in bin_pos_eg_list]

# #Cos binning
# bin_partition_cos_perc = [i/10 for i in range(0, 11, 1)]
# bin_pos_cos_list       = df_GMSH["cthe_cm"].quantile(bin_partition_cos_perc).tolist()
# cos_bins               = [round(i, 4) for i in bin_pos_cos_list]

# #Print
# print(bin_partition_eg_perc)
# print(len(bin_partition_eg_perc))
# print(bin_partition_cos_perc)
# print(len(bin_partition_cos_perc))
# print("epho_bins = ", epho_bins)
# print("len(epho_bins) = ", len(epho_bins))
# print("cos_binss = ", cos_bins)
# print("len(cos_bins) = ", len(cos_bins))

In [ ]:
# #Binning method 2 # of events in each bins (NONEED)
# for i in range(len(epho_bins)-1):
    
#     print("==============================", epho_bins[i], "=============================")
#     for j in range(len(cos_bins)-1):
#         print(len(df_GMSH[(epho_bins[i] <= df_GMSH["epho"])    & (df_GMSH["epho"]    <= epho_bins[i+1]) & \
#                           (cos_bins[j]  <= df_GMSH["cthe_cm"]) & (df_GMSH["cthe_cm"] <= cos_bins[j+1])]))

###### High Energy 

###### Save & Read last Binning 

In [ ]:
# #Save epho
# epho_dict = {"epho_bins": epho_bins}
# df_epho_dict = pd.DataFrame(epho_dict)

# #save to csv
# df_epho_dict.to_csv("./results_Binning/epho_bins")

In [ ]:
# #Save cos_bins dict
# with open("./results_Binning/cos_bins", "w") as f:
#     writer = csv.writer(f, delimiter="\t")
    
#     for key in sorted(cos_bins.keys()):
#         writer.writerow([key] + cos_bins[key])

In [ ]:
#Read
df_epho_dict = pd.read_csv("./results_Binning/epho_bins", index_col=0)
#Into epho_bins list
epho_bins = df_epho_dict[df_epho_dict.columns[0]].tolist()

print(epho_bins)

In [ ]:
#Read cos_bins dict
df       = pd.read_csv("./results_Binning/cos_bins", index_col=0, delimiter="\t", header=None)
cos_bins = df.transpose().to_dict(orient="list")

#Print
for i in sorted(cos_bins.keys()):
    for j in range(len(cos_bins[i])):
        cos_bins[i] = [round(a, 4) for a in cos_bins[i]]
    print(i, cos_bins[i])

### Drop unused columns + Rename 

###### Drop 

In [ ]:
df_GMSH

In [ ]:
#Unwatned columns list
columns_list = ['scPdHt', 'beta_cm', 'gamma_cm','pz_cm', 'p_cm_abs', 'theta_cm', "T_pred", "ice_pred", "phi_sec", \
                "el_pc_pz"]

#Unwatned columns list
# columns_list = ["TRIGBITS", "runNum"]

#Unwanted column list
print(columns_list)

In [ ]:
#Remove
df_GMSH.drop(columns=columns_list, inplace=True)

###### Rename

In [ ]:
df_GMSH.columns

In [ ]:
df_GMSH.rename(columns={"el_px":"px", "el_py":"py", "el_pz":"pz", "el_p_abs":"p_abs", "el_E":"E", "el_mmsq_pi0":"mmsq_pi0", \
                        "el_beta":"beta"}, inplace=True)

### Reduce df memory 

In [ ]:
df_GMSH.memory_usage().sum() / 1000000000

In [ ]:
#Reduce memory usage by changing dtype
def reduce_mem_usage(df):

    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
#         if (col_type == "category"):
#             continue

        if (col_type != object):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

In [ ]:
for i in df_GMSH.columns:
    print(i, "\t",df_GMSH[i].dtype)
    print(df_GMSH[i].dtype.__class__)

In [ ]:
#Change CategoricalDtype to int
df_GMSH["pid"] = df_GMSH["pid"].astype("int")

In [ ]:
#Apply reduce_mem_usage fn.
df_GMSH = reduce_mem_usage(df_GMSH)